In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from profile_generation import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
# pd.set_option('display.max_rows', None)

In [5]:
input_file = 'Retrospective Liver Transplant Data.xlsx'
rows_to_skip = 17

# Get list of patients/sheet names
list_of_patients = get_sheet_names(input_file)

# Define lists
list_of_patient_df = []
list_of_cal_pred_df = []
patients_to_exclude_linear = []
patients_to_exclude_quad = []

for patient in list_of_patients:
        
    df = pd.read_excel(input_file, sheet_name=patient, skiprows=rows_to_skip)
    
    df = clean_data(df, patient)
    df = keep_ideal_data(df, patient, list_of_patient_df)
        
    # Choose and keep data for calibration and efficacy-driven dosing
    cal_pred_linear, patients_to_exclude_linear = cal_pred_data(df, patient, patients_to_exclude_linear, 1)
    cal_pred_quad, patients_to_exclude_quad = cal_pred_data(df, patient, patients_to_exclude_quad, 2)
    
    # Keep patient data with sufficient dose-response pairs and predictions
    cal_pred, list_of_cal_pred_df = keep_target_patients(patient, patients_to_exclude_linear, patients_to_exclude_quad, 
                                                     cal_pred_linear, cal_pred_quad, list_of_cal_pred_df)
    
    

# Print patients to exclude        
patients_to_exclude_linear = sorted(set(patients_to_exclude_linear))
patients_to_exclude_quad = sorted(set(patients_to_exclude_quad))
print(f"Patients to exclude for linear methods: {patients_to_exclude_linear}")
print(f"Patients to exclude for quad methods: {patients_to_exclude_quad}")

# Join dataframes from individual patients
df = pd.concat(list_of_patient_df)
df.reset_index(inplace=True, drop=True)
cal_pred = pd.concat(list_of_cal_pred_df)

## for 131, remove first row in keep_ideal_data function

84    day response dose patient       type
0  2.0      2.4  0.5      84     linear
1  3.0      2.8  1.0      84     linear
2  4.0      3.2  1.5      84     linear
3  5.0      3.1  1.5      84     linear
4  6.0      7.9  3.0      84     linear
5  7.0       10  3.0      84     linear
6  8.0     10.3  3.0      84     linear
0  2.0      2.4  0.5      84  quadratic
1  3.0      2.8  1.0      84  quadratic
2  4.0      3.2  1.5      84  quadratic
3  5.0      3.1  1.5      84  quadratic
4  6.0      7.9  3.0      84  quadratic
5  7.0       10  3.0      84  quadratic
6  8.0     10.3  3.0      84  quadratic
114      day response dose patient       type
0    5.0      4.6  2.0     114     linear
1    6.0      6.2  2.5     114     linear
2    7.0     10.8  3.0     114     linear
3    8.0      5.8  3.0     114     linear
4    9.0     11.3  3.5     114     linear
5   10.0      7.5  2.0     114     linear
6   11.0      6.8  2.0     114     linear
7   12.0        3  1.0     114     linear
8   13.0      4